# [Keras + Universal Sentence Encoder = Deep Meter](https://www.dlology.com/blog/keras-meets-universal-sentence-encoder-transfer-learning-for-text-data/) 

This notebook creates an autoencoder using the Universal Sentence Encoder. The autoencoder output is CMUdict syllables. The dataset is that subset of Allison Parrish's Project Gutenberg poetry archive which happens to scan in iambic pentameter.

The notebook is based on Chengwei Zhang's example of wrapping the USE inside a larger tensorflow model saves to a Keras model (without save the USE itself in the TF model).

The Universal Sentence Encoder makes getting sentence level embeddings as easy as it has historically been to lookup the embeddings for individual words. The sentence embeddings can then be trivially used to compute sentence level meaning similarity as well as to enable better performance on downstream classification tasks using less supervised training data.

Since there are 10 one-hot values for 10 sets of 6k syllables, this is "multi-label classification"
Changes for multi-label classification:
sigmoid activation instead of softmax
binary_crossentropy

Text format is tab-separated, 2 columns: first text, second multi-level
array of syllables:


# Getting Started

This section sets up the environment for access to the Universal Sentence Encoder on TF Hub and provides examples of applying the encoder to words, sentences, and paragraphs.

In [1]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
%cd /content
!git clone https://github.com/LanceNorskog/deep_meter || true
%cd deep_meter
!git pull
# could not figure out how to read gzipped files as text!
!gunzip -qf blobs/*.gz || true
!gunzip -qf prepped_data/*.gz || true

/content
Cloning into 'deep_meter'...
remote: Enumerating objects: 225, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 225 (delta 121), reused 161 (delta 60), pack-reused 0
Receiving objects: 100% (225/225), 20.72 MiB | 8.55 MiB/s, done.
Resolving deltas: 100% (121/121), done.
/content/deep_meter
Already up to date.
gzip: blobs/*.gz: No such file or directory


In [2]:
# boilerplate from base notebook
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import keras.layers as layers
from keras.models import Model
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
np.random.seed(10)

Using TensorFlow backend.


In [0]:
# github deep_meter code
import utils
# should not need this to use utils.flatten but is true anyway?
from itertools import chain
import subprocess
import syllables
# misc for this notebook
from ast import literal_eval

import scipy



In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [5]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/3'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/3'.


In [0]:
# read classified poetry lines: text tab [['syll', 'la', 'ble'], ...]
# clip to only most common syllables with syllable manager
def get_data(filename, syll_mgr, num_symbols):
    lines = open(filename, 'r').read().splitlines()
    text_lines = []
    enc_array = []
    num_syllables = syll_mgr.get_size()
    for i in range(0, len(lines)):
      parts = lines[i].split("\t")
      label = utils.flatten(literal_eval(parts[1]))
      if len(label) != num_symbols:
        continue
      enc = np.zeros((num_symbols * syll_mgr.get_size()), dtype=np.int8)
      for j in range(num_symbols):
        enc[num_syllables * j + syll_mgr.get_encoding(label[j])] = 1
      text_lines.append([parts[0]])
      enc_array.append(enc)

    return (np.array(text_lines), np.array(enc_array))


In [0]:
# syllables in descending order of occurrence - 6k in gutenberg.iambic_pentameter, 15k total
# clamp to most common 100 syllables while debugging- use NCE to get all syllables or interesting number
# 98 + pause + wildcard
num_syllables = 1800 
# iambic pentameter
num_symbols = 10
syll_mgr = syllables.syllables(num_syllables)


In [0]:
(train_text, train_label) = get_data('prepped_data/gutenberg.iambic_pentameter.train', syll_mgr, num_symbols)
num_training = len(train_text)
#train_text = train_text[0:10000]
#train_label = train_label[0:10000]

(test_text, test_label) = get_data('prepped_data/gutenberg.iambic_pentameter.test', syll_mgr, num_symbols)
num_testing = len(test_text)


In [0]:

# slow
num_epochs = 100
adam_0001 = tf.train.AdamOptimizer(0.00003)

## Wrap embed module in a Lambda layer
Explicitly cast the input as a string

In [0]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]


## Assemble model

In [11]:
# changed accuracy from 'choose your own accuracy'
input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = layers.Lambda(UniversalEmbedding, output_shape=(embed_size,), name='TF-Hub')(input_text)
dense = layers.Dense(768, activation='relu', name='Convoluted')(embedding)
dense = layers.Dense(1024, activation='relu', name='Medium')(dense)
dense = layers.Dense(2048, activation='relu', name='Smooth')(dense)
pred = layers.Dense(num_syllables * num_symbols, activation='sigmoid')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='binary_crossentropy', optimizer=adam_0001, metrics=['binary_accuracy'])
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
TF-Hub (Lambda)              (None, 512)               0         
_________________________________________________________________
Convoluted (Dense)           (None, 768)               393984    
_________________________________________________________________
Medium (Dense)               (None, 1024)              787456    
_________________________________________________________________
Smooth (Dense)               (None, 2048)              2099200   
_________________________________________________________________
dense_1 (Dense)              (None, 18000)             36882000  
Total params: 40,162,640
Trainable params: 40,162,640

In [12]:
test_label.shape

(4200, 18000)

In [13]:
print(test_text[200])
syll_mgr.interpret2(test_label[200])

['In this: whatever other blunders lie']
[67, 224, 213, 65, 47, 36, 280, 0, 487, 901]
['IH N', 'DH IH S', 'W AH T', 'EH', 'V ER', 'AH', 'DH ER', '?', 'D ER Z', 'L AY']


## Train Keras model and save weights
This only trains and save our Keras layers not the embed module' weights.

In [14]:
use_saved_model=True
retrain=False
if not use_saved_model or not os.path.exists('./model.h5'):
  with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    if retrain:
      model.load_weights('./model.h5')  
    else:
      session.run(tf.tables_initializer())
    history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),
            epochs=num_epochs,
            callbacks = [EarlyStopping(patience=2)],
            batch_size=32)
    model.save_weights('./model.h5')


Train on 58498 samples, validate on 4200 samples
Epoch 1/100
58498/58498 [==============================] - 122s 2ms/step - loss: 0.0318 - binary_accuracy: 0.9960 - val_loss: 0.0034 - val_binary_accuracy: 0.9994
Epoch 2/100
58498/58498 [==============================] - 116s 2ms/step - loss: 0.0033 - binary_accuracy: 0.9994 - val_loss: 0.0033 - val_binary_accuracy: 0.9994
Epoch 3/100
58498/58498 [==============================] - 116s 2ms/step - loss: 0.0033 - binary_accuracy: 0.9994 - val_loss: 0.0033 - val_binary_accuracy: 0.9994
Epoch 4/100
58498/58498 [==============================] - 115s 2ms/step - loss: 0.0033 - binary_accuracy: 0.9994 - val_loss: 0.0033 - val_binary_accuracy: 0.9995
Epoch 5/100
58498/58498 [==============================] - 115s 2ms/step - loss: 0.0033 - binary_accuracy: 0.9995 - val_loss: 0.0033 - val_binary_accuracy: 0.9995
Epoch 6/100
58498/58498 [==============================] - 119s 2ms/step - loss: 0.0032 - binary_accuracy: 0.9995 - val_loss: 0.0032 - v

In [15]:
!ls -alh | grep model.h5

-rw-r--r-- 1 root root 154M Oct 26 19:37 model.h5


## Make predictions

In [0]:

#new_text = np.array(new_text, dtype=object)[:, np.newaxis]
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h5')  
  predicts = model.predict(test_text, batch_size=32)

In [17]:
first = predicts[0]
max_d = -1000000
min_d = 1000000
max_i = 0
min_i = 0
for i in range(len(predicts) - 1):
  v = predicts[i]
  d = scipy.spatial.distance.cosine(v, first)
  if max_d < d:
    max_d = d
    max_i = i
  if min_d > d:
    min_d = d
    min_i = i
print("Min and max distances: {0} and {1}".format(min_d, max_d))
print(test_text[max_i])
syll_mgr.interpret2(predicts[max_i])

Min and max distances: 2.4950421217972973e-07 and 0.924818860938364
['His fathers blessing from an elder son:']
[10, 571, 280, 571, 280, 40, 10, 571, 972, 180]
['HH IH Z', 'F AA', 'DH ER', 'F AA', 'DH ER', 'AH V', 'HH IH Z', 'F AA', 'DH ER Z', 'S AH N']


In [18]:
for i in range(10,100):
  print(test_text[i])
  syll_mgr.interpret2(predicts[i])
  print('')

['Of humble seeming: and, behind them all,']
[40, 1514, 7, 0, 7, 1514, 7, 0, 153, 0]
['AH V', 'HH AH M', 'AE N D', '?', 'AE N D', 'HH AH M', 'AE N D', '?', 'DH EH M', '?']

['Behind his head upon the shoulders lay,']
[36, 936, 10, 1436, 7, 445, 10, 0, 10, 445]
['AH', 'N IY TH', 'HH IH Z', 'N EH K', 'AE N D', 'HH EH D', 'HH IH Z', '?', 'HH IH Z', 'HH EH D']

['Discover countries, with a kindred heart']
[30, 0, 31, 0, 0, 7, 36, 0, 117, 417]
['T UW', '?', 'DH AH', '?', '?', 'AE N D', 'AH', '?', 'SH AH N Z', 'W ER L D']

['Was open in his praise, and plaudits rose']
[156, 1219, 23, 0, 7, 7, 10, 0, 10, 0]
['W AA Z', 'S IY V D', 'T AH D', '?', 'AE N D', 'AE N D', 'HH IH Z', '?', 'HH IH Z', '?']

['A spear the hero bore of wondrous strength,']
[36, 166, 40, 685, 40, 91, 36, 0, 18, 0]
['AH', 'M AY', 'AH V', 'S AO R D', 'AH V', 'S P IH R', 'AH', '?', 'T AH L', '?']

['Were things indifferent to the Shepherds thoughts.']
[156, 0, 31, 0, 766, 0, 31, 0, 31, 24]
['W AA Z', '?', 'DH AH', '?', 'F R A

In [36]:
categories = df_train.label.cat.categories.tolist()
predict_logits = predicts.argmax(axis=1)
print("Categorie: {0}".format(categories))
predict_labels = [categories[logit] for logit in predict_logits]
predict_labels

NameError: ignored

In [0]:

os.remove('./model.h5')